In [ ]:
#http://data.portic.fr/api/ports?param=&shortenfields=false&both_to=false&date=1787
import http.client
from io import StringIO, BytesIO, TextIOWrapper
import json
import pandas as pd
from flask import Flask, render_template

from bokeh.embed import components
from bokeh.plotting import figure
from bokeh.resources import INLINE

from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral6
from bokeh.transform import factor_cmap



app = Flask(__name__)

def getStateForDate(jsonelement, dateparam):
        """
        jsonelement : [{"1749-1815" : "Toscane"},{"1801-1807" : "Royaume d'Étrurie"},{"1808-1814" : "Empire français"}]
        Internal method to output state for 1787 as dateparam
        return name of the state of the period including dateparam 
        state = getStateForDate(json.load(StringIO('{"1749-1815" : "Toscane"}')), 1787)
        return Toscane
        """
        for dates, state in jsonelement.items():
            datesarray = dates.split('-')
            start = datesarray[0]
            end = datesarray[1]
            if (int(start) <= dateparam <= int(end)):
                return state
            
def getporticdata():
    conn = http.client.HTTPConnection("data.portic.fr")
    conn.request("GET", "/api/ports/?shortenfields=false&both_to=false&date=1787")
    r1 = conn.getresponse()
    #print(r1.status, r1.reason)
    #print(r1)

    data1 = r1.read()  # This will return entire content.
    print(type(data1)) #bytes
    b = BytesIO(data1)
    b.seek(0) #Start of stream (the default).  pos should be  = 0;
    data = json.load(b)
    #print(type(data))

    #print(data)
    type(data)

    df = pd.DataFrame(data)

    print(df.shape)

    df.columns

    df.admiralty.unique()

    # Dealing with null values
    df.admiralty.isnull().values.any() #True 
    values = {'admiralty': 'X'}
    df = df.fillna(value=values)
    df.admiralty.isnull().values.any() #False 

    df.admiralty.unique()
    df.admiralty.unique().size #52
    
    
    #https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html
    for elt in df['belonging_states'].unique():
        if elt is None :
            df.loc[df.belonging_states.eq(elt), 'state_1787'] = 'X' 
        else :
            eltjson = json.loads(elt) 
            for k in eltjson :
                state = getStateForDate(k, 1787)
                if state is not None:
                    df.loc[df.belonging_states.eq(elt), 'state_1787'] = state

    ## Check the results
    df[['toponym', 'belonging_states', 'state_1787']]        
    df.shape

    type(df)
    df.loc[0, ['toponym', 'belonging_states', 'state_1787']]


    
    return df


@app.route('/')
def hello_world():
    return 'Hello, World!'

@app.route('/show_port')
def toto():
    # Dump dataframe as html
    df = getporticdata()
    return render_template('hello.html', msg=df.to_html())

@app.route('/bokeh')
def bokeh():
    
    full = getporticdata()
    
    #df = full
    df = full.iloc[0:10] 
    # How many ports by belonging_states ?
    print(df.groupby('state_1787')['ogc_fid'].count())
    df['counts'] = df.groupby('state_1787')['ogc_fid'].count()
    group = df.groupby(['state_1787'])

    df.drop_duplicates(subset=['toponym'], inplace=True)
    
    #print(df['toponym'])
    #print(df['total'])
    source = ColumnDataSource(data=dict(names=df['toponym'], counts=df['total']))
    #source = ColumnDataSource(data=dict(names=df['state_1787'], counts=df['counts']))
    #source = ColumnDataSource(data=dict(names=group['state_1787'], counts=group['ogc_fid'].count()))

    
    #print(data)
    # init a basic bar chart:
    # http://bokeh.pydata.org/en/latest/docs/user_guide/plotting.html#bars
    fig = figure(x_range=df['toponym'], plot_width=1200, plot_height=600, 
                 title="Counts of toponyms citation for ports per state in 1787")

    fig.vbar(x='names', top='counts', width=0.9, source=source, 
             legend_field="names",line_color='white', 
             #factor_cmap('names', palette=Spectral6, factors=df['state_1787']))
             fill_color='navy')
    
    
    fig.xgrid.grid_line_color = None
    fig.y_range.start = 0
    fig.y_range.end = max(df['total'])*1.2
    fig.legend.orientation = "vertical"
    fig.legend.location = "top_center"
    

    # grab the static resources
    js_resources = INLINE.render_js()
    css_resources = INLINE.render_css()

    # render template
    script, div = components(fig)
    html = render_template(
        'index.html',
        plot_script=script,
        plot_div=div,
        js_resources=js_resources,
        css_resources=css_resources,
    )
    return html

if __name__ == '__main__':
    app.run(port=5050) # Permet de lancer le serveur directement depuis python en exécutant le programme



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5050/ (Press CTRL+C to quit)
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
127.0.0.1 - - [08/Nov/2021 15:16:08] "GET /bokeh HTTP/1.1" 200 -


<class 'bytes'>
(997, 12)
state_1787
Espagne            2
France             6
Grande-Bretagne    1
Monaco             1
Name: ogc_fid, dtype: int64
